# Insert Pre-trained baqckbone onto mask2former

In [7]:
import torch
from transformers import FocalNetForImageClassification, FocalNetConfig, Mask2FormerForUniversalSegmentation


model = Mask2FormerForUniversalSegmentation.from_pretrained('facebook/mask2former-swin-tiny-coco-instance')
config = model.config


# Load the pre-trained FocalNet model
focalnet_model = FocalNetForImageClassification.from_pretrained('microsoft/focalnet-tiny')
focalnet_config = focalnet_model.config

In [8]:
model

Mask2FormerForUniversalSegmentation(
  (model): Mask2FormerModel(
    (pixel_level_module): Mask2FormerPixelLevelModule(
      (encoder): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=96, out_features=96, bias=True)
                      (key): Linear(in_features=96, out_features=96, bias=True)
                      (value

In [9]:
focalnet_model

FocalNetForImageClassification(
  (focalnet): FocalNetModel(
    (embeddings): FocalNetEmbeddings(
      (patch_embeddings): FocalNetPatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): FocalNetEncoder(
      (stages): ModuleList(
        (0): FocalNetStage(
          (layers): ModuleList(
            (0): FocalNetLayer(
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (modulation): FocalNetModulation(
                (projection_in): Linear(in_features=96, out_features=195, bias=True)
                (projection_context): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
                (activation): GELU(approximate='none')
                (projection_out): Linear(in_features=96, out_features=96, bias=True)
                (projection_dropout): Dropout(p=0.0, inplace=F

In [11]:
print(model.model.pixel_level_module.encoder)
print(sum(p.numel() for p in model.model.pixel_level_module.encoder.parameters()))

SwinBackbone(
  (embeddings): SwinEmbeddings(
    (patch_embeddings): SwinPatchEmbeddings(
      (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    )
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): SwinEncoder(
    (layers): ModuleList(
      (0): SwinStage(
        (blocks): ModuleList(
          (0-1): 2 x SwinLayer(
            (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attention): SwinAttention(
              (self): SwinSelfAttention(
                (query): Linear(in_features=96, out_features=96, bias=True)
                (key): Linear(in_features=96, out_features=96, bias=True)
                (value): Linear(in_features=96, out_features=96, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): SwinSelfOutput(
                (dense): Linear(in_features=96, out_features=96, bias=True)
   

In [12]:
print(focalnet_model.focalnet)
print(sum(p.numel() for p in focalnet_model.focalnet.parameters()))

FocalNetModel(
  (embeddings): FocalNetEmbeddings(
    (patch_embeddings): FocalNetPatchEmbeddings(
      (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    )
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): FocalNetEncoder(
    (stages): ModuleList(
      (0): FocalNetStage(
        (layers): ModuleList(
          (0): FocalNetLayer(
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (modulation): FocalNetModulation(
              (projection_in): Linear(in_features=96, out_features=195, bias=True)
              (projection_context): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
              (activation): GELU(approximate='none')
              (projection_out): Linear(in_features=96, out_features=96, bias=True)
              (projection_dropout): Dropout(p=0.0, inplace=False)
              (focal_layers): ModuleList(
                (0): Sequential(
   

In [3]:
# Replace the backbone with the FocalNet model
model.model.pixel_level_module.encoder = focalnet_model.focalnet
model.backbone.config = focalnet_config
model.backbone.model_name = 'microsoft/focalnet-tiny'


# freeze

for param in model.backbone.parameters():
    param.requires_grad = False



# other training hyperparameters

# Modify other training hyperparameters as needed
model.config.learning_rate = 0.001
model.config.num_train_epochs = 5



AttributeError: 'FocalNetForImageClassification' object has no attribute 'backbone'